In [23]:
import requests
import json
import pprint
import re
import patoolib

from pathlib import WindowsPath as Path
import shutil
import os
import time


# with open("config.json", "r") as f:
#     cfg = json.load(f)
#     num_cases = cfg['num_cases']


fields = [
#     'files.data_category',
#     'files.data_format',
#     'files.file_id',
#     'files.file_name',
#     'case_id'
    'submitter_id'
    ]

fields = ",".join(fields)

endpt = "https://api.gdc.cancer.gov/cases"

filters = {
    "op": "and",
    "content":[
        {
            'op': 'in',
            'content': {
                'field': 'project.program.name',  # program
                'value': ['TCGA']
            }
        },
        {
            'op': 'in',
            'content': {
                'field': 'project.name',  # project
                # either 'Glioblastoma Multiforme' or 'Brain Lower Grade Glioma'
                'value': ['Glioblastoma Multiforme']
            }
        },
        {
            "op": "in",
            "content":{
                "field": "cases.project.primary_site",  # primary site
                "value": ["Brain"]
            }
        },
#         {
#             "op": "in",
#             "content":{
#                 "field": "demographic.race",  # race
#                 "value": ["White"]
#             }
#         }
    ]
}

# os.chdir("data/download")

params = {
    "filters": filters,
    "fields": fields,
    'sort': 'submitter_id:asc',
    'size': 1000  # should be >= number of hits outputted
    }
js = requests.post(endpt, headers = {"Content-Type": "application/json"}, json = params).json()
print('warnings: ', js['warnings'])
print('number of hits: ', js['data']['pagination']['total'])
# print(json.dumps(js, indent=2))
case_files = {}

warnings:  {}
number of hits:  617


In [24]:
ids = []
for case in js['data']['hits']:
    print(case)
#     ids.append(case['submitter_sample_ids'])

{'submitter_id': 'TCGA-02-0001', 'id': '30a1fe5e-5b12-472c-aa86-c2db8167ab23'}
{'submitter_id': 'TCGA-02-0003', 'id': 'df3c1d61-79c1-43e9-971a-8029497ffeab'}
{'submitter_id': 'TCGA-02-0004', 'id': '3ac8f759-2df0-4ef5-913f-ce05b8dd8937'}
{'submitter_id': 'TCGA-02-0006', 'id': '98714d95-b62e-4f34-9cd1-91542da463eb'}
{'submitter_id': 'TCGA-02-0007', 'id': 'fb75eb9d-05be-4e6b-a515-f39efb15e66c'}
{'submitter_id': 'TCGA-02-0009', 'id': 'c1f590eb-21ed-4939-adc5-0f7e6f485b56'}
{'submitter_id': 'TCGA-02-0010', 'id': '45a58ede-e7a4-424d-851e-e6a4062e46e4'}
{'submitter_id': 'TCGA-02-0011', 'id': '33ae5566-3c72-4cae-b516-1708db090e32'}
{'submitter_id': 'TCGA-02-0014', 'id': '83264f30-94be-4573-b519-fb0b0fce36c3'}
{'submitter_id': 'TCGA-02-0015', 'id': '697afd24-9632-4244-a455-f95b1fa4f0bf'}
{'submitter_id': 'TCGA-02-0016', 'id': 'adc0c9c3-002a-4fab-b148-e162020ad051'}
{'submitter_id': 'TCGA-02-0021', 'id': 'd85fa9be-f7fb-407b-af46-31f495b210d6'}
{'submitter_id': 'TCGA-02-0023', 'id': 'c45e9e8b-35a

In [10]:
ids

[['TCGA-02-0001-01C', 'TCGA-02-0001-10A', 'TCGA-02-0001-01Z'],
 ['TCGA-02-0003-10A', 'TCGA-02-0003-01A', 'TCGA-02-0003-01Z'],
 ['TCGA-02-0004-01Z', 'TCGA-02-0004-10A', 'TCGA-02-0004-01A'],
 ['TCGA-02-0006-10A', 'TCGA-02-0006-01B', 'TCGA-02-0006-01Z'],
 ['TCGA-02-0007-10A', 'TCGA-02-0007-01Z', 'TCGA-02-0007-01A'],
 ['TCGA-02-0009-01A', 'TCGA-02-0009-10A', 'TCGA-02-0009-01Z'],
 ['TCGA-02-0010-10A', 'TCGA-02-0010-01Z', 'TCGA-02-0010-01A'],
 ['TCGA-02-0011-10A', 'TCGA-02-0011-01B', 'TCGA-02-0011-01Z'],
 ['TCGA-02-0014-01Z', 'TCGA-02-0014-01A', 'TCGA-02-0014-10A'],
 ['TCGA-02-0015-01A', 'TCGA-02-0015-01Z', 'TCGA-02-0015-10A'],
 ['TCGA-02-0016-01A', 'TCGA-02-0016-10A', 'TCGA-02-0016-01Z'],
 ['TCGA-02-0021-01Z', 'TCGA-02-0021-10A', 'TCGA-02-0021-01A'],
 ['TCGA-02-0023-10A', 'TCGA-02-0023-01B', 'TCGA-02-0023-01Z'],
 ['TCGA-02-0024-10A', 'TCGA-02-0024-01Z', 'TCGA-02-0024-01B'],
 ['TCGA-02-0025-01Z', 'TCGA-02-0025-01A', 'TCGA-02-0025-10A'],
 ['TCGA-02-0026-01Z', 'TCGA-02-0026-01B', 'TCGA-02-0026